In [ ]:
import argparse
import copy
import sys
import seaborn as sns
from pathlib import Path
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from datetime import datetime
import pandas as pd

sys.path.append("/vol/biomedic3/mb121/causal-contrastive")
from causal_models.train_setup import setup_dataloaders

from sklearn.metrics import roc_auc_score, balanced_accuracy_score

from causal_models.hps import Hparams
from causal_models.hvae import HVAE2
import torch
import numpy as np
import matplotlib.pyplot as plt
from causal_models.trainer import preprocess_batch

from classification.classification_module import ClassificationModule

from causal_models.plotting_utils import (
    MidpointNormalize,
    plot_cxr_grid,
    plot_counterfactual_viz_mimic,
)


def load_vae_and_module(vae_path):
    args = Hparams()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    checkpoint = torch.load(vae_path, map_location=device)
    args.update(checkpoint["hparams"])
    if not hasattr(args, "cond_prior"):
        args.cond_prior = False
    args.device = device
    vae = HVAE2(args).to(args.device)
    vae.load_state_dict(checkpoint["ema_model_state_dict"])
    vae = vae.eval()
    dataloaders = setup_dataloaders(args, cache=False)
    return vae, dataloaders, args


def get_cxr_counterfactuals(
    model,
    batch,
    args,
    s=0.0,
    col="sex",
    plot=True,
    t=0.1,
    u_t=1.0,
):
    model.cuda()
    results = {}
    _pa = torch.cat([batch[k] for k in args.parents_x], dim=1)
    _pa = (
        _pa[..., None, None]
        .repeat(1, 1, *(args.input_res,) * 2)
        .to(args.device)
        .float()
    )

    zs = model.abduct(x=batch["x"].cuda(), parents=_pa.cuda(), t=t)
    if model.cond_prior:
        zs = [zs[j]["z"] for j in range(len(zs))]

    px_loc, px_scale = model.forward_latents(zs, parents=_pa, t=t)

    cf_pa = {k: batch[k] for k in args.parents_x}

    if col == "scanner":
        cf_pa[col] = 1 - cf_pa["scanner"]
        do = {"scanner": cf_pa[col]}
    elif col == "sex":
        cf_pa["sex"] = 1 - cf_pa["sex"]
        do = {"sex": cf_pa[col]}
    else:
        NotImplementedError

    _cf_pa = torch.cat([cf_pa[k] for k in args.parents_x], dim=1)
    _cf_pa = (
        _cf_pa[..., None, None]
        .repeat(1, 1, *(args.input_res,) * 2)
        .to(args.device)
        .float()
    )
    cf_loc, cf_scale = model.forward_latents(zs, parents=_cf_pa.cuda(), t=t)

    diff = cf_loc - px_loc
    diff = diff - diff.mean(dim=[-1, -2], keepdim=True)
    cf_loc = cf_loc + s * diff
    u = (batch["x"] - px_loc) / px_scale.clamp(min=1e-12)
    cf_scale = cf_scale * u_t
    cf_x = torch.clamp(cf_loc + cf_scale * u, min=-1, max=1)
    results["DE"] = cf_x.cpu()
    results["REC"] = px_loc.cpu()
    results["GT"] = batch["x"].cpu()
    if isinstance(col, list):
        results["cf_col_values"] = torch.cat([cf_pa[c] for c in col], 1)
    else:
        results["cf_col_values"] = cf_pa[col]

    if plot:
        f = plot_counterfactual_viz_mimic(
            args,
            batch["x"].cpu(),
            cf_x.cpu(),
            {k: batch[k] for k in args.parents_x},
            cf_pa,
            do,
            results["REC"],
            save=False,
        )
        f.show()

    return results

In [ ]:
from classification.classification_module import ClassificationModule

model_path = "/vol/biomedic3/mb121/causal-contrastive/outputs/sex_scanner/beta3fixed/checkpoint.pt"
vae, dataloader, args = load_vae_and_module(model_path)
test_loader = dataloader["valid"]
run_name = Path(model_path).parent.stem + datetime.now().strftime("%d-%m-%y/%H-%M")
print(run_name)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

model_paths = {
    "sex": "../outputs/run_o856uapk/best.ckpt",
    "scanner": "../outputs/run_o6i8yqna/best.ckpt",
}

for column in ["scanner"]:
    classification_model = ClassificationModule.load_from_checkpoint(
        model_paths[column]
    ).model.eval()
    classification_model.cuda()
    confs = []
    labels = []
    confs_true = []
    true_label = []
    all_feats = []
    true_feats = []
    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            batch = preprocess_batch(args, batch, expand_pa=False)
            out_d = get_cxr_counterfactuals(
                model=vae,
                batch=batch,
                args=args,
                col=column,
                plot=i < 1,
                s=0.0,
                t=0.5,
                u_t=1.0,
            )
            label = out_d["cf_col_values"].numpy()
            inputs = (out_d["DE"].to(args.device) + 1) / 2
            feats = classification_model.get_features(inputs.cuda())
            logits = classification_model.classify_features(feats).cpu()
            feats = feats.cpu()
            all_feats.append(feats)
            probas = torch.softmax(logits, 1).numpy()
            confs.append(probas)
            labels.append(label)
            inputs = (out_d["GT"].to(args.device) + 1) / 2
            feats = classification_model.get_features(inputs.cuda())
            logits = classification_model.classify_features(feats).cpu()
            feats = feats.cpu()
            true_feats.append(feats)
            probas = torch.softmax(logits, 1).numpy()
            confs_true.append(probas)
            true_label.append(batch[column].numpy())
            if i % 50 == 0 and i > 0:
                print(i)
                print(np.concatenate(true_label).mean())
            if i > 50:
                break

    print(f"Variable CF {column}")
    targets = np.concatenate(labels)
    targets_true = np.concatenate(true_label)

    all_feats = np.concatenate(all_feats)
    true_feats = np.concatenate(true_feats)

    clf = Pipeline([("pca", PCA(0.95)), ("rf", RandomForestClassifier(250))])
    print(
        cross_val_score(
            clf,
            X=np.concatenate((all_feats, true_feats)),
            y=np.concatenate(
                (np.ones(all_feats.shape[0]), np.zeros(true_feats.shape[0]))
            ),
        ).mean()
    )

    if targets.shape[1] == 1:
        targets = targets.reshape(-1)
        targets_true = targets_true.reshape(-1)
    else:
        targets = np.argmax(targets, 1)
        targets_true = np.argmax(targets_true, 1)

    confs = np.concatenate(confs)
    confs_true = np.concatenate(confs_true)

    res = {}

    bal_true = balanced_accuracy_score(targets_true, np.argmax(confs_true, 1))
    bal_cf = balanced_accuracy_score(targets, np.argmax(confs, 1))

    print(f"Balanced accuracy GT {bal_true:.3f}")
    print(f"Balanced accuracy CF {bal_cf:.3f}")

    res["run_name"] = run_name
    res["bal_true"] = bal_true
    res["bal_cf"] = bal_cf

    cm_true = confusion_matrix(targets_true, np.argmax(confs_true, 1))
    cm_cf = confusion_matrix(targets, np.argmax(confs, 1))

    res["cm_true"] = cm_true.reshape(-1)
    res["cm_cf"] = cm_cf.reshape(-1)
    res["cm_true"] = ",".join([str(i) for i in res["cm_true"]])
    res["cm_cf"] = ",".join([str(i) for i in res["cm_cf"]])
    res["cf_var"] = column
    ConfusionMatrixDisplay.from_predictions(targets, np.argmax(confs, 1))
    ConfusionMatrixDisplay.from_predictions(
        targets, np.argmax(confs, 1), normalize="true"
    )

    if confs.shape[1] == 2:
        roc_gt = roc_auc_score(targets_true, confs_true[:, 1])
        roc_cf = roc_auc_score(targets, confs[:, 1])
        print(f"ROC-AUC GT {roc_gt:.3f}")
        print(f"ROC-AUC CF {roc_cf:.3f}")
        res["roc_gt"] = roc_gt
        res["roc_cf"] = roc_cf
    else:
        try:
            roc_gt = roc_auc_score(targets_true, confs_true, multi_class="ovr")
            roc_cf = roc_auc_score(targets, confs, multi_class="ovr")
            print(f"ROC-AUC GT {roc_gt:.3f}")
            print(f"ROC-AUC  CF {roc_cf:.3f}")
            res["roc_gt"] = roc_gt
            res["roc_cf"] = roc_cf
        except ValueError:
            pass